In [1]:
from sqlalchemy import create_engine
import pandas as pd 
import json

with open('/home/douglas/postgres_credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']
    address = data['address']

conn = create_engine('postgresql://{}:{}@{}:5432/ex_social'.format(username, password, address))

/home/douglas/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Look at tables

In [2]:
pd.read_sql_query("SELECT * FROM users", conn)

,user_id,first_name,last_name,house
0,1,Harry,Potter,Gryffindor
1,2,Ron,Wesley,Gryffindor
2,3,Hermonie,Granger,Gryffindor
3,4,Ginny,Weasley,Gryffindor
4,5,Draco,Malfoy,Slytherin
5,6,Tom,Riddle,Slytherin
6,7,Luna,Lovegood,Ravenclaw
7,8,Cho,Chang,Ravenclaw
8,9,Cedric,Diggory,Hufflepuff


In [3]:
pd.read_sql_query("SELECT * FROM follows", conn)

,user_id,follows,date_created
0,1,2,1993-09-01
1,2,1,1989-01-01
2,3,1,1993-07-01
3,2,3,1994-10-10
4,3,2,1995-03-01
5,4,2,1988-08-08
6,4,1,1988-08-08
7,1,4,1994-04-02
8,1,5,2000-01-01
9,5,1,2000-01-02


# Problems

In [4]:
# How many users are there in each house?
pd.read_sql_query("SELECT house, count(user_id) FROM users GROUP BY house", conn)

,house,count
0,Ravenclaw,2
1,Slytherin,2
2,Gryffindor,4
3,Hufflepuff,1


In [5]:
# List all following links that were created before September 1st, 1993
pd.read_sql_query("SELECT * FROM follows WHERE date_created < '1993-09-01'", conn)

,user_id,follows,date_created
0,2,1,1989-01-01
1,3,1,1993-07-01
2,4,2,1988-08-08
3,4,1,1988-08-08
4,5,6,1986-01-10
5,7,1,1990-02-02


In [6]:
# List all rows from the follows table, replacing both user_ids with first name. Hint: it may help to make this a VIEW
pd.read_sql_query('''SELECT u1.first_name AS user_first, u2.first_name AS follows_first, f.date_created
FROM follows f 
JOIN users u1 ON f.user_id = u1.user_id
JOIN users u2 ON f.follows = u2.user_id ''', conn)


,user_first,follows_first,date_created
0,Harry,Ron,1993-09-01
1,Ron,Harry,1989-01-01
2,Hermonie,Harry,1993-07-01
3,Ron,Hermonie,1994-10-10
4,Hermonie,Ron,1995-03-01
5,Ginny,Ron,1988-08-08
6,Ginny,Harry,1988-08-08
7,Harry,Ginny,1994-04-02
8,Harry,Draco,2000-01-01
9,Draco,Harry,2000-01-02


In [7]:
conn.execute(''' CREATE OR REPLACE VIEW follows_w_fnames  AS 
SELECT u1.first_name AS user_first, u2.first_name AS follows_first, f.date_created
FROM follows f 
JOIN users u1 ON f.user_id = u1.user_id
JOIN users u2 ON f.follows = u2.user_id''')
# executes 
# OR REPLACE in case view already created 
# IF NOT EXISTS for tables

In [8]:
#confirm view created 
pd.read_sql_query("SELECT * FROM follows_w_fnames LIMIT 5", conn)

,user_first,follows_first,date_created
0,Harry,Ron,1993-09-01
1,Ron,Harry,1989-01-01
2,Hermonie,Harry,1993-07-01
3,Ron,Hermonie,1994-10-10
4,Hermonie,Ron,1995-03-01


In [9]:
# List all the following links established before September 1st 1993, but this time use the users first names.
pd.read_sql_query("SELECT * FROM follows_w_fnames WHERE date_created < '1993-09-01'", conn)

,user_first,follows_first,date_created
0,Ron,Harry,1989-01-01
1,Hermonie,Harry,1993-07-01
2,Ginny,Ron,1988-08-08
3,Ginny,Harry,1988-08-08
4,Draco,Tom,1986-01-10
5,Luna,Harry,1990-02-02


In [10]:
# Give a count of how many people followed each user as of 1999-12-31. Give the result in term of "users full name, number of followers".
pd.read_sql_query("SELECT CONCAT(u.first_name, ' ', u.last_name) AS user_followed_name, \
                  CASE WHEN s.num_followers IS NULL THEN 0 ELSE s.num_followers END AS num_followers \
                  FROM (SELECT follows_first AS first_name, \
COUNT(user_first) AS num_followers \
FROM follows_w_fnames WHERE date_created <= '1999-12-31' GROUP BY 1) s \
RIGHT JOIN users u \
ON s.first_name = u.first_name ", conn)
# case when clause to catch draco with no followers


,user_followed_name,num_followers
0,Harry Potter,5
1,Ron Wesley,3
2,Hermonie Granger,1
3,Ginny Weasley,1
4,Draco Malfoy,0
5,Tom Riddle,1
6,Luna Lovegood,1
7,Cho Chang,3
8,Cedric Diggory,3


In [11]:
# List the number of users each user follows
pd.read_sql_query('''
SELECT CONCAT(u.first_name, ' ', u.last_name) AS user__name,
       CASE WHEN s.num_followed IS NULL THEN 0 ELSE s.num_followed END AS num_followed
FROM users u 
LEFT JOIN (
    SELECT user_first AS first_name,
           COUNT(follows_first) AS num_followed
           FROM follows_w_fnames GROUP BY 1) s
ON s.first_name = u.first_name
''' 
                  , conn)
# LEFT JOIN same as right join, just have to switch table order

,user__name,num_followed
0,Harry Potter,5
1,Ron Wesley,2
2,Hermonie Granger,3
3,Ginny Weasley,3
4,Draco Malfoy,2
5,Tom Riddle,0
6,Luna Lovegood,2
7,Cho Chang,2
8,Cedric Diggory,1


In [12]:
# List all rows from follows where someone from one house follows someone from a different house.
pd.read_sql_query('''
SELECT CONCAT(u1.first_name, ' ', u1.last_name) AS user__name,
u1.house as user_house, 
CONCAT(u2.first_name, ' ', u2.last_name) AS follows_name, 
u2.house AS follows_house
FROM follows_w_fnames f
JOIN users u1 
ON f.user_first = u1.first_name
JOIN users u2 
ON f.follows_first = u2.first_name 
AND u1.house != u2.house''', conn)
# have access to all columns in joins to filter by ON

,user__name,user_house,follows_name,follows_house
0,Harry Potter,Gryffindor,Draco Malfoy,Slytherin
1,Draco Malfoy,Slytherin,Harry Potter,Gryffindor
2,Luna Lovegood,Ravenclaw,Harry Potter,Gryffindor
3,Harry Potter,Gryffindor,Luna Lovegood,Ravenclaw
4,Harry Potter,Gryffindor,Cho Chang,Ravenclaw
5,Cho Chang,Ravenclaw,Harry Potter,Gryffindor
6,Cho Chang,Ravenclaw,Cedric Diggory,Hufflepuff
7,Cedric Diggory,Hufflepuff,Cho Chang,Ravenclaw
8,Hermonie Granger,Gryffindor,Cedric Diggory,Hufflepuff
9,Ginny Weasley,Gryffindor,Cedric Diggory,Hufflepuff


In [13]:
# We define a friendship as a relationship between two users where both follow each other. The friendship is established when the later of the two links is established.
pd.read_sql_query('''
SELECT DISTINCT 
        CASE WHEN f1.date_created > f2.date_created 
            THEN f1.user_first
            ELSE f2.user_first END AS user1,
        CASE WHEN f1.date_created > f2.date_created 
            THEN f1.follows_first
            ELSE f2.follows_first END AS user2, 
        CASE WHEN f1.date_created > f2.date_created 
            THEN f1.date_created
            ELSE f2.date_created END AS friendship_established
FROM follows_w_fnames f1 
JOIN follows_w_fnames f2
ON f1.user_first = f2.follows_first 
AND f2.user_first = f1.follows_first ''', conn)

# WHERE user1 is the one following back
# Is there an easier way to remove duplicate friendships?

,user1,user2,friendship_established
0,Draco,Harry,2000-01-02
1,Harry,Ginny,1994-04-02
2,Harry,Luna,1996-10-01
3,Cedric,Cho,1996-01-10
4,Harry,Ron,1993-09-01
5,Hermonie,Ron,1995-03-01
6,Cho,Harry,1995-09-01


In [22]:
%%time 
# List all unrequited followings (i.e. where A follows B but B does not follow A)
results = pd.read_sql_query('''SELECT 
f2.user_first,
f2.follows_first,
s.array_agg
FROM follows_w_fnames f2 
LEFT JOIN (
SELECT user_first, array_agg(follows_first)
FROM follows_w_fnames 
GROUP BY user_first) s
ON f2.follows_first = s.user_first

 ''', conn)

CPU times: user 2.73 ms, sys: 809 µs, total: 3.54 ms
Wall time: 5.54 ms


In [23]:
def check_unrequited(row):
    '''Check if user is in the followee's list of follows
    Returns (bool)
    '''
    if row['array_agg']:
        if row['user_first'] in row['array_agg']:
            return False
        else:
            return True
    else:
        return True
    
# python function more intuitive


In [24]:
%%time 
results['Unrequited'] = results.apply(check_unrequited, axis=1)

CPU times: user 5.08 ms, sys: 303 µs, total: 5.38 ms
Wall time: 6.16 ms


In [17]:
results

,user_first,follows_first,array_agg,Unrequited
0,Harry,Ron,"[Harry, Hermonie]",False
1,Ron,Harry,"[Ron, Ginny, Draco, Luna, Cho]",False
2,Hermonie,Harry,"[Ron, Ginny, Draco, Luna, Cho]",True
3,Ron,Hermonie,"[Harry, Ron, Cedric]",False
4,Hermonie,Ron,"[Harry, Hermonie]",False
5,Ginny,Ron,"[Harry, Hermonie]",True
6,Ginny,Harry,"[Ron, Ginny, Draco, Luna, Cho]",False
7,Harry,Ginny,"[Ron, Harry, Cedric]",False
8,Harry,Draco,"[Harry, Tom]",False
9,Draco,Harry,"[Ron, Ginny, Draco, Luna, Cho]",False


In [25]:
%%time 
results[results['Unrequited']]

CPU times: user 0 ns, sys: 880 µs, total: 880 µs
Wall time: 890 µs


,user_first,follows_first,array_agg,Unrequited
2,Hermonie,Harry,"[Ron, Ginny, Draco, Luna, Cho]",True
5,Ginny,Ron,"[Harry, Hermonie]",True
10,Draco,Tom,None,True
17,Luna,Cho,"[Harry, Cedric]",True
18,Hermonie,Cedric,[Cho],True
19,Ginny,Cedric,[Cho],True


In [20]:
%%time 
# SQL only version
# 1. concat follows to itself, but flipping the user_id and follows 
# 2. count and keep only ones with count of 1 since they would be duplicated if followed back
# 3. join result back to follows to keep only rows from original table and not the flipped rows
pd.read_sql_query('''
SELECT d.user_id, d.follows
FROM
    (SELECT s.user_id, s.follows, count(s.user_id) 
    FROM (
        SELECT * FROM follows
        UNION ALL (SELECT follows AS user_id, user_id AS follows, date_created 
                    FROM follows)) s 
    GROUP BY s.user_id, s.follows
    HAVING count(s.user_id) = 1) d
JOIN follows f
ON d.user_id = f.user_id
AND d.follows = f.follows
                
                ''', conn)

CPU times: user 3.31 ms, sys: 0 ns, total: 3.31 ms
Wall time: 10.4 ms


,user_id,follows
0,3,1
1,3,9
2,4,2
3,4,9
4,5,6
5,7,8
